In [1]:
from ase import io
import pickle
import numpy as np

In [2]:
logs2read = pickle.load(open("/home/michael/PycharmProjects/gaussian_job_manage/complete_logs.p", "rb"))

In [3]:
init_struct = io.read(logs2read[0].replace(".log", ".com"))
final_struct = io.read(logs2read[0], format='gaussian-out')

In [4]:
logs_to_check = []
names2check = []
for log in logs2read:
    print("Reading: {}".format(log))
    try:
        atoms_obj = io.read(log.replace(".log", ".com"))
    except IndexError:
        "Error reading {}, check manually".format(log.replace(".log", ".com"))
        logs_to_check.append(log)
        names2check.append(log.split("/")[4])
        continue
    init_struct = atoms_obj.get_positions()

    if init_struct[0][0] != 0.0:
        print("Check structure for {}, this run was a continuation".format(log))
        logs_to_check.append(log)
        names2check.append(log.split("/")[4])
        continue
    final_struct = io.read(log, format='gaussian-out').get_positions()
    atomic_nums = atoms_obj.get_atomic_numbers()
    origin_i = init_struct[np.argmax(atomic_nums)]
    origin_f = final_struct[np.argmax(atomic_nums)]
    init_distances = []
    final_distances = []
    for xyz_i, xyz_f in zip(init_struct, final_struct):
        init_distances.append(np.sqrt((xyz_i[0]-origin_i[0])**2+(xyz_i[1]-origin_i[1])**2+(xyz_i[2]-origin_i[2])**2))
        final_distances.append(np.sqrt((xyz_f[0]-origin_f[0])**2+(xyz_f[1]-origin_f[1])**2+(xyz_f[2]-origin_f[2])**2))
        
    num_neighbours = 6
    if "VAC" in log:
        if "31a" in log or "3a1" in log or "32a" in log or "41a" in log:
            num_neighbours = 5

    nearest_neighbours_i = np.argpartition(init_distances, num_neighbours)
    nearest_neighbours_i = [nn for nn in nearest_neighbours_i[:num_neighbours+1] if nn!=np.argmax(atomic_nums)]
    nearest_neighbours_f = np.argpartition(final_distances, num_neighbours)
    nearest_neighbours_f = [nn for nn in nearest_neighbours_f[:num_neighbours+1] if nn!=np.argmax(atomic_nums)]
    if set(nearest_neighbours_i)!=set(nearest_neighbours_f):
        print(nearest_neighbours_i)
        print(nearest_neighbours_f)

        print("**********************************")
        print("Atoms coordinating the metal have changed: {}".format(log))
        print("**********************************")
        logs_to_check.append(log)
        names2check.append(log.split("/")[4])
    else:
        # all the indices are the same for init and final, just check the bond length change
        for index in nearest_neighbours_i:
            if final_distances[index] > 1.2*init_distances[index]:
                print(index)
                print("Bond distance 20% increase for {}".format(log))
                logs_to_check.append(log)
                names2check.append(log.split("/")[4])

Reading: /home/michael/ht_scaling_results/Fe_31a_17/VAC/HS/Fe_31a_17_VAC_HS.log
Reading: /home/michael/ht_scaling_results/Fe_31a_17/OH/HS/Fe_31a_17_OH_HS.log
Reading: /home/michael/ht_scaling_results/Fe_31a_17/OOH/HS/Fe_31a_17_OOH_HS.log
Reading: /home/michael/ht_scaling_results/Co_31a_23/VAC/HS/Co_31a_23_VAC_HS.log
Reading: /home/michael/ht_scaling_results/Co_31a_23/OH/LS/Co_31a_23_OH_LS.log
Reading: /home/michael/ht_scaling_results/Co_31a_23/OOH/LS/Co_31a_23_OOH_LS.log
Reading: /home/michael/ht_scaling_results/Cr_32a_35/VAC/HS/back/Cr_32a_35_VAC_HS-b_3e-1.log
Check structure for /home/michael/ht_scaling_results/Cr_32a_35/VAC/HS/back/Cr_32a_35_VAC_HS-b_3e-1.log, this run was a continuation
Reading: /home/michael/ht_scaling_results/Cr_32a_35/OH/HS/Cr_32a_35_OH_HS.log
Reading: /home/michael/ht_scaling_results/Cr_32a_35/OOH/HS/Cr_32a_35_OOH_HS-fromOH.log
Check structure for /home/michael/ht_scaling_results/Cr_32a_35/OOH/HS/Cr_32a_35_OOH_HS-fromOH.log, this run was a continuation
Reading:

Check structure for /home/michael/ht_scaling_results/Fe_penta_2/OH/LS/Fe_penta_2_OH_LS.log, this run was a continuation
Reading: /home/michael/ht_scaling_results/Fe_penta_2/OOH/Fe_penta_2_OOH_HS.log
Check structure for /home/michael/ht_scaling_results/Fe_penta_2/OOH/Fe_penta_2_OOH_HS.log, this run was a continuation
Reading: /home/michael/ht_scaling_results/Mn_31a_18/VAC/HS/Mn_31a_18_VAC_HS.log
Reading: /home/michael/ht_scaling_results/Mn_31a_18/OH/HS/Mn_31a_18_OH_HS.log
Reading: /home/michael/ht_scaling_results/Mn_31a_18/OOH/HS/Mn_31a_18_OOH_HS.log
Reading: /home/michael/ht_scaling_results/Fe_31a_20/VAC/HS/Fe_31a_20_VAC_HS.log
Reading: /home/michael/ht_scaling_results/Fe_31a_20/OH/HS/Fe_31a_20_OH_HS.log
Reading: /home/michael/ht_scaling_results/Fe_31a_20/OOH/LS/Fe_31a_20_OOH_LS.log
Reading: /home/michael/ht_scaling_results/Co_31a_6_/VAC/HS/back/Co_31a_6_VAC_HS-b_3e-1.log
Check structure for /home/michael/ht_scaling_results/Co_31a_6_/VAC/HS/back/Co_31a_6_VAC_HS-b_3e-1.log, this run was

Reading: /home/michael/ht_scaling_results/Fe_3a1_26/OH/HS/Fe_3a1_26_OH_HS.log
[16, 12, 34, 18, 3, 10]
[3, 16, 12, 34, 10, 17]
**********************************
Atoms coordinating the metal have changed: /home/michael/ht_scaling_results/Fe_3a1_26/OH/HS/Fe_3a1_26_OH_HS.log
**********************************
Reading: /home/michael/ht_scaling_results/Fe_3a1_26/OOH/HS/Fe_3a1_26_OOH_HS-retry.log
Check structure for /home/michael/ht_scaling_results/Fe_3a1_26/OOH/HS/Fe_3a1_26_OOH_HS-retry.log, this run was a continuation
Reading: /home/michael/ht_scaling_results/Ni_penta_1/VAC/HS/Ni_penta_1_VAC_HS.log
Check structure for /home/michael/ht_scaling_results/Ni_penta_1/VAC/HS/Ni_penta_1_VAC_HS.log, this run was a continuation
Reading: /home/michael/ht_scaling_results/Ni_penta_1/OH/LS/Ni_penta_1_OH-LS.log
Check structure for /home/michael/ht_scaling_results/Ni_penta_1/OH/LS/Ni_penta_1_OH-LS.log, this run was a continuation
Reading: /home/michael/ht_scaling_results/Ni_penta_1/OOH/Ni_penta_1_OOH-LS.l

Reading: /home/michael/ht_scaling_results/Ni_31a_2_/OH/LS/Ni_31a_2_OH_LS.log
Reading: /home/michael/ht_scaling_results/Ni_31a_2_/OOH/LS/Ni_31a_2_OOH_LS.log
Reading: /home/michael/ht_scaling_results/Co_oct_desorbed_4_/Co_pbp_4_VAC_HS.log
Reading: /home/michael/ht_scaling_results/Co_oct_desorbed_4_/OH/LS/Co_oct_desorbed_4_OH_LS.log
Reading: /home/michael/ht_scaling_results/Co_oct_desorbed_4_/OOH/LS/Co_oct_desorbed_4_OOH_LS.log
Reading: /home/michael/ht_scaling_results/Cr_3a1_26/VAC/HS/Cr_3a1_26_VAC_HS.log
Reading: /home/michael/ht_scaling_results/Cr_3a1_26/OH/HS/Cr_3a1_26_OH_HS.log
Reading: /home/michael/ht_scaling_results/Cr_3a1_26/OOH/HS/Cr_3a1_26_OOH_HS.log
Reading: /home/michael/ht_scaling_results/Ru_penta_1/VAC/LS/Ru_penta_1_VAC_LS.log
Check structure for /home/michael/ht_scaling_results/Ru_penta_1/VAC/LS/Ru_penta_1_VAC_LS.log, this run was a continuation
Reading: /home/michael/ht_scaling_results/Ru_penta_1/OH/LS/Ru_penta_1_OH_LS.log
Check structure for /home/michael/ht_scaling_resu

45
Bond distance 20% increase for /home/michael/ht_scaling_results/Cr_3a1_16/VAC/HS/Cr_3a1_16_VAC_HS.log
Reading: /home/michael/ht_scaling_results/Cr_3a1_16/OH/HS/back/Cr_3a1_16_OH_HSb-cartesian-b_3e-1.log
Check structure for /home/michael/ht_scaling_results/Cr_3a1_16/OH/HS/back/Cr_3a1_16_OH_HSb-cartesian-b_3e-1.log, this run was a continuation
Reading: /home/michael/ht_scaling_results/Cr_3a1_16/OOH/HS/Cr_3a1_16_OOH_HS.log
Reading: /home/michael/ht_scaling_results/Mn_oct_desorbed_5_/VAC/Mn_pbp_5_VAC_HSb.log
Check structure for /home/michael/ht_scaling_results/Mn_oct_desorbed_5_/VAC/Mn_pbp_5_VAC_HSb.log, this run was a continuation
Reading: /home/michael/ht_scaling_results/Mn_oct_desorbed_5_/OH/HS/Mn_oct_desorbed_5_OH_HS.log
[18, 44, 25, 27, 3, 17]
[18, 44, 25, 27, 3, 26]
**********************************
Atoms coordinating the metal have changed: /home/michael/ht_scaling_results/Mn_oct_desorbed_5_/OH/HS/Mn_oct_desorbed_5_OH_HS.log
**********************************
Reading: /home/mich

FileNotFoundError: [Errno 2] No such file or directory: '/home/michael/ht_scaling_results/Co_hexa_4/VAC/tpssh-Co-27-VAC-doublet-freq.com'

In [ ]:
set(names2check)

In [ ]:
logs_without_unneeded = [x for x in logs_to_check if "penta" not in x and "hexa" not in x and "pbp" not in x and "forw" not in x and "back" not in x and "b.log" not in x]
logs_without_unneeded